# Gapfilling a metabolic model

In [1]:
import cobra
#If you have CPLEX, uncomment this
#import cplex
import cobrakbase
#import modelseedpy.fbapkg
from modelseedpy import GapfillingPkg, KBaseMediaPkg
from modelseedpy import FBAHelper, MSBuilder

cobrakbase 0.2.8


In [2]:
kbase_api = cobrakbase.KBaseAPI()
model = kbase_api.get_from_ws("test_model",18528)
#If you have CPLEX, uncomment this
#model.solver = 'optlang-cplex'
template = kbase_api.get_from_ws("GramNegModelTemplateV3","NewKBaseModelTemplates")
media = kbase_api.get_from_ws("Carbon-D-Glucose","KBaseMedia")
model = MSBuilder.gapfill_model(model,"bio1",template,media)

kmp = KBaseMediaPkg(model)
kmp.build_package(media)
sol=model.optimize()
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00009_e0,EX_cpd00009_e0,0.9998,0,0.00%
cpd00013_e0,EX_cpd00013_e0,6.037,0,0.00%
cpd00027_e0,EX_cpd00027_e0,5,6,100.00%
cpd00030_e0,EX_cpd00030_e0,0.00639,0,0.00%
cpd00034_e0,EX_cpd00034_e0,0.00639,0,0.00%
cpd00048_e0,EX_cpd00048_e0,0.1755,0,0.00%
cpd00058_e0,EX_cpd00058_e0,0.00639,0,0.00%
cpd00063_e0,EX_cpd00063_e0,0.00639,0,0.00%
cpd00067_e0,EX_cpd00067_e0,61.85,0,0.00%
cpd00099_e0,EX_cpd00099_e0,0.00639,0,0.00%


In [85]:
from modelseedpy.helpers import get_template
from modelseedpy.core.mstemplate import MSTemplateBuilder
template_core = get_template('template_core')
template_core = MSTemplateBuilder.from_dict(template_core).build()
template_gramneg = get_template('template_gram_neg')
compcompounds = []
for o in template_gramneg['compcompounds']:
    a, b = o['id'].split('_')
    if b not in ['c', 'e']:
        #print(o)
        pass
    else:
        compcompounds.append(o)
template_gramneg['compcompounds'] = compcompounds
template_gramneg = MSTemplateBuilder.from_dict(template_gramneg).build()

In [17]:
for template_reaction in template.reactions:
    template_reaction

In [19]:
template_reaction.id.split("_")[0]

'rxn09295'

In [76]:
template_gramneg = get_template('template_gram_neg')

In [52]:
from modelseedpy import MSBuilder
from cobra.core import Model, Reaction, Metabolite
def build_full_model(template, index='0'):
    model = Model('modelo')
    model.add_reactions([r.to_reaction(index=index) for r in template.reactions])
    exchanges = MSBuilder.build_exchanges(model, 'e0')
    #biomasses = MSBuilder.build_biomasses(model, template, '')
    #print(exchanges)
    bio_atp = Reaction('bio2', 'bio2', '', -10000, 10000)
    bio_atp.add_metabolites({
        model.metabolites.cpd00067_c0: 1,
        model.metabolites.cpd00002_c0: -1,
        model.metabolites.cpd00008_c0: 1,
        model.metabolites.cpd00001_c0: -1,
        model.metabolites.cpd00009_c0: 1,
    })
    model.add_reactions(exchanges)
    model.add_reactions([bio_atp])
    model.objective = 'bio2'
    #model.add_reactions(biomasses)
    return model

model_core = build_full_model(template_core)
#model_gramneg = build_full_model(template_gramneg)

In [87]:
def extend_model_with_template_metabolites(model, template, index='0'):
    """
    Add all missing template compartment compounds to the model
    :param template:
    :param index:
    :return:
    """
    new_metabolites = {}
    new_exchange = []
    new_demand = []
    for template_compound in template.compcompounds:
        compartment = template_compound.compartment
        compartment_index = "0" if compartment == 'e' else index
        cobra_metabolite = GapfillingPkg.convert_template_compound(None, template_compound, compartment_index, template)  # TODO: move function out
        if cobra_metabolite.id not in model.metabolites and cobra_metabolite.id not in new_metabolites:
            new_metabolites[cobra_metabolite.id] = cobra_metabolite
            #self.model.add_metabolites([cobra_metabolite])
            msid = FBAHelper.modelseed_id_from_cobra_metabolite(cobra_metabolite)
            if msid in ["cpd02701", "cpd11416", "cpd15302"]:
                if msid != "cpd11416" or cobra_metabolite.compartment == "c0":
                    new_demand.append(cobra_metabolite.id)
            if compartment == "e":
                new_exchange.append(cobra_metabolite.id)
    # Adding all metabolites to model prior to adding reactions
    #self.model.add_metabolites(new_metabolites.values())

    return new_exchange, new_demand, new_metabolites
new_exchange, new_demand, new_metabolites = extend_model_with_template_metabolites(model_core, template_gramneg)

In [ ]:
model_gramneg.medium = {
    'EX_cpd00001_e0': 1000,
    'EX_cpd00067_e0': 1000,
    'EX_cpd00009_e0': 1000,
    'EX_cpd00011_e0': 1000,
    'EX_cpd00029_e0': 3,
}

In [11]:
model_core.medium = {
    'EX_cpd00001_e0': 1000,
    'EX_cpd00067_e0': 1000,
    'EX_cpd00009_e0': 1000,
    'EX_cpd00011_e0': 1000,
    'EX_cpd00027_e0': 3,
}
model_core.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00027_e0,EX_cpd00027_e0,3,6,100.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00011_e0,EX_cpd00011_e0,-6,1,33.33%
cpd00363_e0,EX_cpd00363_e0,-6,2,66.67%


In [22]:
model_gramneg.optimize()

,fluxes,reduced_costs
rxn00594_c,0.0,0.0
rxn01087_c,0.0,0.0
rxn03234_c,0.0,-0.0
rxn01734_c,0.0,0.0
rxn02282_c,0.0,0.0
...,...,...
EX_cpd00561_e,-0.0,0.0
EX_cpd03453_e,-0.0,0.0
EX_cpd18072_e,-0.0,0.0
EX_cpd18074_e,-0.0,0.0


In [89]:
from modelseedpy import FBAHelper
from modelseedpy import GapfillingPkg
pkg_gf = GapfillingPkg(model_core)
pkg_gf.build(template_gramneg)

In [95]:
model_core.medium = {
    'EX_cpd00001_e0': 1000,
    'EX_cpd00067_e0': 1000,
    'EX_cpd00009_e0': 1000,
    'EX_cpd00011_e0': 1000,
    'EX_cpd00029_e0': 3,
}
sol = model_core.optimize()

In [98]:
for r in model_core.reactions:
    r.lower_bound = -1000
    r.upper_bound = 1000

In [103]:
model_core.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


In [102]:
sol = model_core.optimize()
sol

,fluxes,reduced_costs
rxn01975_c0,1000.00000,0.0
rxn00265_c0,93.63949,0.0
rxn00549_c0,1000.00000,-0.0
rxn01123_c0,1000.00000,-0.0
rxn14427_c0,1000.00000,-0.0
...,...,...
EX_cpd00561_e0,-0.00000,0.0
EX_cpd03453_e0,0.00000,0.0
DM_cpd11416_c0,-0.00000,0.0
DM_cpd15302_c0,-0.00000,0.0


In [100]:
model_core.objective = 'bio2'

In [48]:
len(model_core.reactions)


9975

In [51]:
for rxn_id in pkg_gf.gapfilling_penalties:
    print(rxn_id, pkg_gf.gapfilling_penalties[rxn_id])

rxn00594_c0 {'forward': 4, 'added': 1}
rxn01087_c0 {'forward': 4, 'added': 1}
rxn03234_c0 {'reverse': 5, 'forward': 5, 'added': 1}
rxn01734_c0 {'reverse': 4, 'forward': 4, 'added': 1}
rxn02282_c0 {'reverse': 4, 'forward': 4, 'added': 1}
rxn10561_c0 {'reverse': 7, 'forward': 7, 'added': 1}
rxn02201_c0 {'forward': 2, 'added': 1}
rxn02241_c0 {'forward': 4, 'added': 1}
rxn09471_c0 {'forward': 6, 'added': 1}
rxn05704_c0 {'reverse': 7, 'forward': 7, 'added': 1}
rxn00351_c0 {'forward': 2, 'added': 1}
rxn02637_c0 {'reverse': 6, 'forward': 6, 'added': 1}
rxn10496_c0 {'reverse': 7, 'forward': 7, 'added': 1}
rxn05994_c0 {'reverse': 5, 'forward': 7.654999999999999, 'added': 1}
rxn07431_c0 {'reverse': 2, 'forward': 2, 'added': 1}
rxn11746_c0 {'forward': 5, 'added': 1}
rxn00836_c0 {'reverse': 2, 'added': 1}
rxn03805_c0 {'reverse': 4, 'forward': 4, 'added': 1}
rxn02209_c0 {'reverse': 5, 'forward': 5, 'added': 1}
rxn08977_c0 {'reversed': 1, 'reverse': 3.2560000000000002}
rxn00390_c0 {'reverse': 4.5237

In [47]:
model_core.metabolites.cpd00006_0

Metabolite identifier,cpd00006_0
Name,NADP
Memory address,0x07f5224dfb2b0
Formula,C21H25N7O17P3
Compartment,0
In 0 reaction(s),


In [44]:
import re
index_hash = {"none": 0}
for metabolite in model_core.metabolites:
    print(metabolite.id)
    if re.search('_[a-z]\d+$', metabolite.id) is not None:
        m = re.search('_([a-z])(\d+)$', metabolite.id)
        if m[1] != "e":
            if m[2] not in index_hash:
                index_hash[m[2]] = 0
            index_hash[m[2]] += 1
    else:
        index_hash["none":0]
        # Iterating over all indecies with more than 10 intracellular compounds:

cpd00006_c0
cpd00863_c0
cpd00005_c0
cpd00067_c0
cpd00911_c0
cpd00137_c0
cpd00029_c0
cpd00032_c0
cpd00001_c0
cpd00290_c0
cpd00009_c0
cpd00072_c0
cpd00002_c0
cpd00176_c0
cpd00008_c0
cpd02711_c0
cpd00209_c0
cpd18074_c0
cpd00067_e0
cpd00075_c0
cpd18072_c0
cpd00024_c0
cpd00056_c0
cpd00011_c0
cpd03189_c0
cpd00079_c0
cpd00106_e0
cpd00106_c0
cpd00171_c0
cpd00047_c0
cpd11225_c0
cpd00007_c0
cpd00100_c0
cpd00100_e0
cpd00139_c0
cpd00040_c0
cpd00013_c0
cpd15560_c0
cpd15561_c0
cpd00022_c0
cpd00120_c0
cpd00010_c0
cpd03123_c0
cpd00891_c0
cpd00519_c0
cpd00102_c0
cpd00238_c0
cpd00236_c0
cpd00011_e0
cpd01476_c0
cpd10515_c0
cpd00028_c0
cpd00003_c0
cpd01947_c0
cpd00004_c0
cpd00361_c0
cpd00055_c0
cpd03583_c0
cpd00203_c0
cpd03049_c0
cpd00020_c0
cpd00260_c0
cpd00036_c0
cpd00078_c0
cpd00095_c0
cpd00159_e0
cpd00159_c0
cpd00876_c0
cpd00287_c0
cpd11620_c0
cpd11621_c0
cpd00198_c0
cpd00331_c0
cpd00130_c0
cpd15414_c0
cpd00104_c0
cpd00029_e0
cpd00971_e0
cpd00971_c0
cpd00027_c0
cpd00027_e0
cpd00169_c0
cpd00080_c0
cpd0

TypeError: unhashable type: 'slice'